# Prep 1: steps to install the packages

In [1]:
!pip install py_entitymatching
!pip install scipy
!pip install numpy
!pip install pandas

# [NEED MODIFICATION: Modify this cell to point to the file location]
# Prep 2: enter the file location on your harddisk

In [2]:
table_a = 'TableA'
table_b = 'TableB'
candidate_set = 'CandidateSet'
prediction_set = 'PredictionList'

# Prep 3: reading the files into pandas dataframe

In [3]:
import pandas as pd
from scipy.stats import norm
from numpy import sqrt
dfa = pd.read_csv(table_a)
dfb = pd.read_csv(table_b)
dfc = pd.read_csv(candidate_set)
dfp = pd.read_csv(prediction_set)

# Module: debug_blocker
# Description: debug the blocking rule using the below script to ensure you are not dropping true matches
# Note: You need to run Prep 1 and 2 in order to run this

In [4]:
# Example input format:
# Format of table_a:
# _id, attribute1, attribute2, ....., attributen

# Format of table_b:
# _id, attribute1, attribute2, ....., attributen

# Format of candidate_set
# A_id,B_id
# where A_id is _id from table_a and B_id is the _id column value from table_b

In [5]:
import py_entitymatching as em
import pandas as pd

def run_debug_blocker(table_a, table_b, table_a_key, table_b_key, candidate_set):
    dfl = em.read_csv_metadata(table_a, key=table_a_key)
    dfr = em.read_csv_metadata(table_b, key=table_b_key)

    # reading the candidate set and adding key
    dfcand = pd.read_csv(candidate_set)
    print(len(dfcand))
    dfcand.drop_duplicates(inplace=True)
    dfcand.to_csv('cand_set_with_index.csv', index_label='id')

    dfcset = em.read_csv_metadata('cand_set_with_index.csv', key='id', ltable=dfl, 
                                  rtable=dfr, fk_ltable='A_id', fk_rtable='B_id')

    # running debug blocker to identify the records in A x B \ C
    debug_file = em.debug_blocker(dfcset, dfl, dfr)
    
    return debug_file

In [6]:
candidate_set = 'ReducedCandidateSet.csv'
debug_file = run_debug_blocker(table_a, table_b, '_id', '_id', candidate_set)

Metadata file is not present in the given path; proceeding to read the csv file.
Metadata file is not present in the given path; proceeding to read the csv file.
Metadata file is not present in the given path; proceeding to read the csv file.


1520


In [7]:
debug_file

,_id,ltable__id,rtable__id,ltable_title,ltable_authors,ltable_book_format,ltable_cur_price,ltable_old_price,ltable_publisher,ltable_publication_date,...,ltable_Up_System,rtable_title,rtable_authors,rtable_book_format,rtable_cur_price,rtable_old_price,rtable_publisher,rtable_publication_date,rtable_product_dimensions,rtable_Up_System
0,0,2188,4077,The Coldest Winter Ever: A Novel,Sister Souljah,Paperback,$14.96,$17.00,Washington Square Press; Reprint edition,09/20/05,...,CM,The House at Riverton: A Novel,Kate Morton,Paperback,$15.30,$17.00,Washington Square Press,03/03/2009,8.24(w) x 5.30(h) x 1.32(d),CM
1,1,4153,4429,"The John Wayne Code: Wit, Wisdom and Timeless Advice",Media Lab Books and Editors of the Official John Wayne Magazine,Paperback,$10.87,$15.99,Media Lab Books; Lea edition,05/09/17,...,CM,The Official John Wayne Family Cookbook: Recipes and Recollections from Duke's Kitchen to Yours,Official John Wayne Magazine (Editor),Paperback,$20.69,$22.99,Media Lab Books,10/18/2016,7.90(w) x 8.90(h) x 0.70(d),CM
2,2,915,3751,"The Swamp: The Everglades, Florida, and the Politics of Paradise",Michael Grunwald,Paperback,$10.44,$18.00,Simon & Schuster; Reprint edition,03/27/07,...,CM,"The Conquerors: Roosevelt, Truman and the Destruction of Hitler's Germany, 1941-1945",Michael Beschloss,Paperback,$16.20,$18.00,Simon & Schuster,09/30/2003,6.12(w) x 9.25(h) x 1.20(d),CM
3,3,4372,1657,Queen Bey: A Celebration of the Power and Creativity of Beyonc Knowles-Carter,Veronica Chambers,Hardcover,$16.05,$27.99,St. Martin's Press,03/05/19,...,CM,Here's the Catch: A Memoir of the Miracle Mets and More,Ron Swoboda,Hardcover,$19.59,$27.99,St. Martin's Press,06/11/2019,6.12(w) x 9.25(h) x (d),CM
4,4,213,5433,Team of Rivals: The Political Genius of Abraham Lincoln,Doris Kearns Goodwin,Paperback,$14.28,$22.00,Simon & Schuster,09/26/06,...,CM,"Wrestling With His Angel: The Political Life of Abraham Lincoln Vol. II, 1849-1856",Sidney Blumenthal,Paperback,$16.20,$18.00,Simon & Schuster,05/22/2018,6.10(w) x 9.10(h) x 1.60(d),CM
5,5,2583,375,American War: A Novel,Omar El Akkad,Paperback,$11.52,$16.95,Vintage; Reprint edition,01/30/18,...,CM,American War,Omar El Akkad,Paperback,$15.26,$16.95,Knopf Doubleday Publishing Group,01/30/2018,5.10(w) x 7.80(h) x 1.10(d),CM
6,6,1483,5023,House on Fire: A Novel,Bonnie Kistler,Hardcover,$17.66,$27.00,Atria Books,03/12/19,...,CM,This Tender Land: A Novel,William Kent Krueger,Hardcover,$18.90,$27.00,Atria Books,09/03/2019,6.00(w) x 9.00(h) x (d),CM
7,7,1814,5414,The Only Woman in the Room: A Novel,Marie Benedict,Hardcover,$18.19,$25.99,Sourcebooks Landmark,01/08/19,...,CM,Woman 99: A Novel,Greer Macallister,Hardcover,$23.39,$25.99,Sourcebooks,03/05/2019,6.10(w) x 9.10(h) x 1.40(d),CM
8,8,1725,5213,The Nightingale: A Novel,Kristin Hannah,Paperback,$9.94,$16.99,St. Martin's Griffin; Reprint edition,04/25/17,...,CM,Victoria: A Novel,Daisy Goodwin,Paperback,$15.29,$16.99,St. Martin's Press,09/26/2017,5.50(w) x 8.20(h) x 1.10(d),CM
9,9,1220,4264,The Little Paris Bookshop: A Novel,Nina George,Paperback,$11.96,$16.00,Broadway Books; Reprint edition,03/22/16,...,CM,The Little French Bistro,Nina George,Paperback,$14.40,$16.00,Crown/Archetype,03/06/2018,5.10(w) x 7.90(h) x 1.00(d),CM


# Module: estimate_precision_recall
# Description: the below code helps you get an estimation of P/R on the candidate set.

In [8]:
import pandas as pd
from scipy.stats import norm
from numpy import sqrt

delta = .05
Z = norm.ppf(1 - (delta / 2))

def estimate_PR(labeled_pairs, reduced_cands, predicted_matches):
    '''
    labeled_pairs - a pandas dataframe with schema id1,id2,label
                    Note label needs to be Boolean

    reduced_cands - a pandas dataframe with schema id1,id2
    predicted_matches - a pandas dataframe with schema id1,id2
    
    return:
        ( (recall lower bound, recall upper bound), (precision lower bound, precision upper bound) )
    '''

    labeled_pairs.drop_duplicates(inplace=True)
    labeled_pairs.columns = ['id1', 'id2', 'label']
    reduced_cands.columns = ['id1', 'id2']
    reduced_cand_set = set(zip(reduced_cands.id1, reduced_cands.id2))
    predicted_matches = set(zip(predicted_matches.id1, predicted_matches.id2))
    
    # estimate the recall
    # number of positives in the labeled sample
    actual_pos = float(labeled_pairs.label.sum())
    # the maximum number of postives in the candidate set
    max_actual_pos = float(actual_pos + len(reduced_cand_set) - len(labeled_pairs))
    
    # true positives in the labeled sample
    true_pos = float(labeled_pairs.apply(lambda x : (x['id1'], x['id2']) in predicted_matches and x['label'], axis=1).sum())
    #estimated recall
    recall = float(true_pos / actual_pos)

    recall_error = Z * sqrt( ((recall * (1 - recall)) / (actual_pos)) * ((max_actual_pos - actual_pos) / (max_actual_pos - 1)) )


    # estimate Precision
    labeled_set  = set(zip(labeled_pairs.id1, labeled_pairs.id2))
    predicted_pos = float(len(labeled_set & predicted_matches))
    
    predicted_pos_in_reduced_cand_set = float(len(reduced_cand_set & predicted_matches))
    
    alpha =  predicted_pos_in_reduced_cand_set / len(predicted_matches)
    precision = alpha * (true_pos / predicted_pos)
    
    precision_error = alpha * Z * sqrt( ((precision * (1 - precision)) / predicted_pos) * (float((len(predicted_matches) - predicted_pos)) / (len(predicted_matches)  - 1)) )

    return ((recall - recall_error, recall + recall_error),
            (precision - precision_error, precision + precision_error))

# [NEED MODIFICATION: Modify this cell to point to the file location]

In [9]:
# read the labeled pairs file, i.e. the file with the labels
labeled_pairs = pd.read_csv('labeled_pairs.csv')
print(estimate_PR(labeled_pairs, dfc, dfp))

((0.96213812423464351, 1.0019337320527815), (0.95664452530551392, 0.99573642707543841))


# Alternatively if you run into issues running this script on your laptop you can use
# https://colab.research.google.com/notebooks/welcome.ipynb